In [16]:
import numpy as np
import pandas as pd
import requests
import re
import os
import json

In [17]:
df = pd.read_csv('P3_GrantExport.csv', sep=';', index_col=0, na_values=['Nicht zuteilbar - NA'])

In [18]:
df.index.is_unique

True

In [19]:
df.University.describe()

count                       48393
unique                         76
top       Universität Zürich - ZH
freq                         6774
Name: University, dtype: object

In [20]:
def trim_university_name(full_name):
    match = re.match('^(.+?)(?: - .+)?$', full_name)
    if match is None:
        return None
    else:
        return match.group(1)

In [21]:
def get_api_key():
    with open('google_api_key.json') as f:
        json_data = json.load(f)
        return json_data['api_key']

In [164]:
def search_place(place_name):
    BASE_URL = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
    
    api_key = get_api_key()
    parameters = {'key': api_key, 'query': place_name}
    
    r = requests.get(BASE_URL, params=parameters)
    response = r.json()

    if response['status'] != 'OK':
        return None
    else:
        result = response['results'][0]
        location = result['geometry']['location']
        place_id = result['place_id']
        
        return (place_id, location['lat'], location['lng'])

In [28]:
def search_canton(place_id):
    BASE_URL = 'https://maps.googleapis.com/maps/api/geocode/json'
    
    api_key = get_api_key()
    parameters = {'key': api_key, 'place_id': place_id}
    
    r = requests.get('https://maps.googleapis.com/maps/api/geocode/json', params=parameters)
    response = r.json()
    
    if response['status'] != 'OK':
        return None
    else:
        result = response['results'][0]
        
        for component in result['address_components']:
            if 'administrative_area_level_1' in component['types']:
                return component['short_name']
            
        return None

In [167]:
def search_uni(university_name):
    place = search_place(university_name)
    if place is None:
        return None
    else:
        place_id, lat, lng = place
        canton = search_canton(place_id)
        return (canton, lat, lng)

In [168]:
uni_names = df.University.drop_duplicates().dropna()

In [169]:
uni_infos = {uni: search_uni(trim_university_name(uni)) for uni in uni_names}

In [170]:
uni_data = uni_names.map(lambda s: uni_infos[s] if uni_infos[s] is not None else (np.nan,) * 3)
unis = pd.DataFrame(uni_data.tolist(), columns=['Canton', 'Latitude', 'Longitude'], index=uni_names)

In [171]:
unis.count()

Canton       60
Latitude     60
Longitude    60
dtype: int64

In [172]:
unis.to_csv('universities.csv')

In [159]:
unis[unis.Canton.isnull()]

,Canton,Latitude,Longitude
University,,,
"NPO (Biblioth., Museen, Verwalt.) - NPO",NaN,NaN,NaN
"Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL",NaN,NaN,NaN
Université de Neuchâtel - NE,NaN,NaN,NaN
ETH Zürich - ETHZ,NaN,NaN,NaN
Inst. de Hautes Etudes Internat. et du Dév - IHEID,NaN,NaN,NaN
Universität St. Gallen - SG,NaN,NaN,NaN
Weitere Institute - FINST,NaN,NaN,NaN
Firmen/Privatwirtschaft - FP,NaN,NaN,NaN
Pädagogische Hochschule Graubünden - PHGR,NaN,NaN,NaN
